Задания
Повторим pandas-исследования. По возможности всю предобработку данных доверьте SQL.

В скрипте на Питоне:

Создайте и заполните таблицу запросами, создайте техническое поле ID c параметрами INTEGER PRIMARY KEY AUTOINCREMENT.
Добавьте индекс на поле salary. Изменится ли после этого размер файла? На сколько?
Выведите количество записей.
Выведите количество мужчин и женщин.
У скольки записей заполены skills?
Получить заполненные скиллы.
Вывести зарплату только у тех, у кого в скилах есть Python.
Построить перцентили и разброс по з/п у мужчин и женщин.
Построить графики распределения по з/п мужчин и женщин (а также в зависимости от высшего образования).

#### 1.Создайте и заполните таблицу запросами, создайте техническое поле ID c параметрами INTEGER PRIMARY KEY AUTOINCREMENT.


In [ ]:
import sqlite3 
import os
import numpy as np
import os.path
import re
import pandas as pd
import matplotlib.pyplot as plt
import re

In [ ]:
con = sqlite3.connect('works.sqlite')

Создаем таблицу, в первом столбце которой будет расположен уникальный индентификатор

In [ ]:
cur = con.cursor()
cur.execute("DROP TABLE IF EXISTS works")
cur.execute('CREATE TABLE works ('
               'ID INTEGER PRIMARY KEY AUTOINCREMENT,'
               'salary INTEGER,'
               'educationType TEXT,'
               'jobTitle TEXT,'
               'qualification TEXT,'
               'gender TEXT,'
               'dateModify TEXT,'
               'skills TEXT,'
               'otherInfo TEXT)')

Получим описание столбцов всей таблицы с помощь запроса PRAGMA table_info()

In [ ]:
cur.execute("pragma table_info(works)").fetchall()

С помощью Pandas запишем всю информацию из датафрейма в sql

In [ ]:
data = pd.read_csv("works.csv")
data.to_sql('works', con, if_exists="append", index=False)

#### 2.Добавьте индекс на поле salary. Изменится ли после этого размер файла? На сколько?

In [ ]:
weight_before = os.path.getsize(r'works.sqlite')
print(f"Размер до создания индекса: {weight_before}.")

In [106]:
cur.execute("create index salary_index on works (salary)")

OperationalError: database is locked

In [ ]:
weight_after= os.path.getsize(r'works.sqlite')
print(f"Размер после создания индекса: {weight_after} байт(а).")
print(f"Разница: {weight_before - weight_after} байт(а).")

Вывод: После добавления индекса размер файла не меняется

#### 3.Выведите количество записей.

In [ ]:
print(cur.execute("SELECT COUNT(*) FROM works").fetchone())

Количество записей 32683 

#### 4.Выведите количество мужчин и женщин.

In [ ]:
count = cur.execute("SELECT gender, count(*) FROM works GROUP BY works.gender").fetchall()
count

#### 4.У скольки записей заполены skills?

In [ ]:
skill_not_null = cur.execute("SELECT COUNT(*) FROM works WHERE skills IS NOT NULL").fetchone()
skill_not_null

In [ ]:
В столбце skills заполнено 8972 строк

#### 5.Получить заполненные скиллы.

In [ ]:
skills = cur.execute("SELECT skills FROM works WHERE skills IS NOT NULL").fetchall()
print(skills)


#### 6.Вывести зарплату только у тех, у кого в скилах есть Python.

In [ ]:
python_skills = cur.execute("SELECT salary FROM works WHERE lower(skills) LIKE '%python%' OR lower(skills) LIKE '%питон%' LIKE '%пайтон%'").fetchall()
python_skills


#### 7.Построить перцентили и разброс по з/п у мужчин и женщин

In [ ]:
q = [i/10 for i in range(1, 11)]
for gender in genders:
    query = f"SELECT salary FROM works WHERE {gender=}"
    salaries = [slr[0] for slr in cur.execute(query).fetchall()]
    res = np.quantile(np.array(salaries), q)
    print(f"{gender=}")
    print(*[f"{i} -> {j}" for i, j in zip(q, res)], sep="\n")

#### 8.Построить графики распределения по з/п мужчин и женщин (а также в зависимости от высшего образования).

In [ ]:


gender_edu = cur.execute("SELECT DISTINCT educationType FROM works WHERE educationType IS NOT NULL").fetchall()
gender_edu = [ed[0] for ed in educations]

for gender in genders:
    for education in gender_edu:
        plt.figure(figsize=(10, 5))  
        plt.grid()
        plt.title(f"Распределение по заработной плате для пол : '{gender}' с образованием: '{education}'") 
        plt.xlabel("Зарплата") 
        plt.ylabel("Количество") 
        query = f'SELECT salary FROM works WHERE {gender=} and {educationType=}'
        salaries = [i[0] for i in cur.execute(query).fetchall()]
        plt.hist(salaries, bins=25)
        

# ДЗ

Выделим отдельные сущности:

Создайте отдельную таблицу с гендером, заполните ее значениями, сделайте на нее внешний ключ из таблицы works.

Отдельная таблица для образования.

#### Создайте отдельную таблицу с гендером, заполните ее значениями, сделайте на нее внешний ключ из таблицы works.

Создаем новую базу таблицу и сохраним изменения

In [107]:
сonn = sqlite3.connect('works.sqlite')
cursor = сonn.cursor()
cursor.execute('PRAGMA foreign_keys = true')

Заполним таблицу

In [ ]:
cursor.execute('DROP TABLE IF EXISTS works')
cursor.execute('CREATE TABLE works ('
               'ID INTEGER PRIMARY KEY AUTOINCREMENT,'
               'salary INTEGER,'
               'educationType TEXT,'
               'jobTitle TEXT,'
               'qualification TEXT,'
               'gender TEXT,'
               'dateModify TEXT,'
               'skills TEXT,'
               'otherInfo TEXT)')


In [ ]:
df = pd.read_csv("works.csv")

Создадим функцию для возврата необходимых тегов

In [ ]:
def return_tags(value):
    return re.sub(r'<[^>]*>', '', str(value))

Очистим таблицу от тегов и перезапишем ее

In [ ]:
df['skills'] = df['skills'].apply(return_tags)
df['otherInfo'] = df['otherInfo'].apply(return_tags)
df.to_sql("works", con, if_exists='append', index=False)

 Создадим и заполненим словари genders и educations

In [ ]:
cursor.execute('DROP TABLE IF EXISTS genders')
cursor.execute('CREATE TABLE genders(genderName TEXT PRIMARY KEY )')
cursor.execute('INSERT INTO genders SELECT DISTINCT gender FROM works WHERE gender IS NOT NULL')
cursor.execute('DROP TABLE IF EXISTS educations')
cursor.execute('CREATE TABLE educations(educationType TEXT PRIMARY KEY )')
cursor.execute('INSERT INTO educations SELECT DISTINCT educationType FROM works WHERE works.educationType IS NOT NULL')

#### Отдельная таблица для образования.

In [ ]:
cursor.execute('CREATE TABLE new_works ('
               'ID INTEGER PRIMARY KEY AUTOINCREMENT,'
               'salary INTEGER,'
               'educationType TEXT REFERENCES educations(educationType) ON DELETE CASCADE ON UPDATE CASCADE,'
               'jobTitle TEXT,'
               'qualification TEXT,'
               'gender TEXT REFERENCES genders(genderName) ON DELETE CASCADE ON UPDATE CASCADE,'
               'dateModify TEXT,'
               'skills TEXT,'
               'otherInfo TEXT)')
cursor.execute('INSERT INTO new_works SELECT * FROM works')
cursor.execute('DROP TABLE works')
cursor.execute('ALTER TABLE new_works RENAME TO works')
con.commit()